# Create the Bayesian Network

In [ ]:
from pgmpy.models import BayesianNetwork
import pandas as pd

data = pd.read_csv("./dataset_cleaned.csv")

network_structure = [
    # (from, to),
    # (from, to),
    # (from, to)
]

BN = BayesianNetwork(network_structure).fit(data)

# Explore some features

In [2]:
...

Ellipsis

# Queries

In [3]:
...

Ellipsis